In [1]:
import pandas as pd

# Boston Condo Conversion Cleaning

This file will take the Boston area's parcel data and convert it into information about condo conversions that occurred in the Boston area between 2015 and 2024

## Initial Cleaning

In this section, we'll read in the csv files and convert them into a format that will make our analysis easier

In [2]:
years = range(2015, 2025)
parcel_data = {year: pd.read_csv(f"parcel_data/data_{year}.csv") for year in years}

/var/folders/77/mfz4g0815pzbmgddh8q96whm0000gn/T/ipykernel_18828/2606240460.py:2: DtypeWarning: Columns (60,63) have mixed types. Specify dtype option on import or set low_memory=False.
  parcel_data = {year: pd.read_csv(f"parcel_data/data_{year}.csv") for year in years}
/var/folders/77/mfz4g0815pzbmgddh8q96whm0000gn/T/ipykernel_18828/2606240460.py:2: DtypeWarning: Columns (60,63) have mixed types. Specify dtype option on import or set low_memory=False.
  parcel_data = {year: pd.read_csv(f"parcel_data/data_{year}.csv") for year in years}
/var/folders/77/mfz4g0815pzbmgddh8q96whm0000gn/T/ipykernel_18828/2606240460.py:2: DtypeWarning: Columns (15,60,63) have mixed types. Specify dtype option on import or set low_memory=False.
  parcel_data = {year: pd.read_csv(f"parcel_data/data_{year}.csv") for year in years}
/var/folders/77/mfz4g0815pzbmgddh8q96whm0000gn/T/ipykernel_18828/2606240460.py:2: DtypeWarning: Columns (15,60) have mixed types. Specify dtype option on import or set low_memory=Fa

In [3]:
parcel_data[2020].head()

,PID,CM_ID,GIS_ID,ST_NUM,ST_NAME,ST_NAME_SUF,UNIT_NUM,ZIPCODE,PTYPE,LU,...,U_BTH_STYLE2,U_BTH_STYLE3,U_KITCH_TYPE,U_KITCH_STYLE,U_HEAT_TYP,U_AC,U_FPLACE,U_INT_FIN,U_INT_CND,U_VIEW
0,100271002,100271000.0,100271000,356,PRINCETON,ST,1,2128.0,102,CD,...,NaN,NaN,F - Full Eat In,M - Modern,W - Ht Water/Steam,N - None,0.0,N - Normal,G - Good,A - Average
1,100271004,100271000.0,100271000,356,PRINCETON,ST,2,2128.0,102,CD,...,NaN,NaN,F - Full Eat In,M - Modern,W - Ht Water/Steam,N - None,0.0,N - Normal,G - Good,A - Average
2,100271006,100271000.0,100271000,356,PRINCETON,ST,3,2128.0,102,CD,...,NaN,NaN,F - Full Eat In,M - Modern,W - Ht Water/Steam,N - None,0.0,N - Normal,G - Good,A - Average
3,100272000,NaN,100272000,352,PRINCETON,ST,NaN,2128.0,105,R3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100273000,NaN,100273000,,PRINCETON,ST,NaN,2128.0,132,RL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# While the 2015 - 2020 data has suffix separate from the street name, 2021 - 2024 data combines them
# So, we'll put the street names from the 2015 - 2020 data into the same format as the 2021 - 2024 data
for year in range(2015, 2021):
    parcel_data[year]['ST_NAME'] = parcel_data[year]['ST_NAME'] + " " + parcel_data[year]["ST_NAME_SUF"].fillna("")
    parcel_data[year] = parcel_data[year].drop(columns=["ST_NAME_SUF"])

In [5]:
# A list of columns that will probably be unnecessary for data analysis. If you'd like to play around with this, feel free to remove some.
# Note that some of these columns also only exist in one or a few years' data
columns_to_drop = [
    'BTHRM_STYLE1', 'BTHRM_STYLE2', 'BTHRM_STYLE3', 'FIRE_PLACE', 'FIREPLACES', 'full_address', 
    'HEAT_FUEL', 'HEAT_SYSTEM', 'KITCHEN_STYLE1', 'KITCHEN_STYLE2', 'KITCHEN_STYLE3', 'LATITUDE', 
    'Location', 'LONGITUDE', 'MAIL CS', 'MAIL_ADDRESS', 'MAIL_ADDRESSEE', 'MAIL_CITY', 'MAIL_STATE', 
    'MAIL_STREET_ADDRESS', 'MAIL_ZIP_CODE', 'MAIL_ZIPCODE', 'ORIENTATION', 'OWNER MAIL ADDRESS', 'OWNER_MAIL_ADDRESS', 
    'OWNER_MAIL_CS', 'OWNER_MAIL_ZIPCODE', 'PLUMBING', 'PROP_VIEW', 'R_BLDG_STYL', 'R_BTH_STYLE', 'R_BTH_STYLE2', 
    'R_BTH_STYLE3', 'R_FPLACE', 'R_INT_FIN', 'R_KITCH_STYLE', 'R_KITCH_STYLE2', 'R_KITCH_STYLE3', 'R_VIEW', 'ROOF_COVER', 
    'ROOF_STRUCTURE', 'S_BLDG_STYL', 'SFYI_VALUE', 'U_BTH_STYLE', 'U_BTH_STYLE2', 'U_BTH_STYLE3', 'U_CORNER', 'U_FPLACE', 
    'U_KITCH_STYLE', 'U_ORIENT', 'U_VIEW', 'OWNER', 'OWN_OCC', 'INT_WALL', 'R_ROOF_TYP', 'U_KITCH_TYPE', 'KITCHEN_TYPE', 'KITCHENS', 
    'R_KITCH', 'EXT_FNISHED', 'EXT_FINISHED', 'R_EXT_FIN', 'U_INT_FIN', 'S_EXT_FIN', 'AC_TYPE', 'U_BASE_FLOOR', 'CORNER_UNIT', 'U_AC', 
    'R_AC', 'U_BDRMS', 'U_FULL_BTH', 'U_HALF_BTH', 'U_HEAT_TYP', 'U_INT_CND', 'U_NUM_PARK', 'U_TOT_RMS'
]

In [6]:
# Some years have small differences in naming. Convert this naming for better consistency
for year in years:
    parcel_data[year] = (parcel_data[year]
                         .rename(columns={'YR_REMOD': 'YR_REMODEL', 
                                          'ZIP_CODE': 'ZIPCODE', 
                                          'KITCHEN': 'KITCHENS', 
                                          'EXT_FNISHED': 'EXT_FINISHED', 
                                          'U_KIT_TYPE': 'U_KITCH_TYPE',
                                          'S_UNIT_RC': 'RC_UNITS',
                                          'S_UNIT_RES': 'RES_UNITS',
                                          'S_UNIT_COM': 'COM_UNITS',
                                          ' GROSS_TAX ': 'GROSS_TAX',
                                          'AV_LAND': 'LAND_VALUE',
                                          'AV_BLDG': 'BLDG_VALUE',
                                          'AV_TOTAL': 'TOTAL_VALUE',
                                          'R_BDRMS': 'BED_RMS',
                                          'R_FULL_BTH': 'FULL_BTH',
                                          'HLF_BATH': 'HALF_BTH',
                                          'R_HALF_BTH': 'HALF_BTH',
                                          'R_HEAT_TYP': 'HEAT_TYP',
                                          'R_TOTAL_RMS': 'TT_RMS'})
                         .drop(columns=columns_to_drop, errors="ignore"))

In [7]:
# Convert all the street numbers to string to make for consistent comparison. Some empty street numbers
# were stored as NaN's, so change those to empty strings
for year in years:
    parcel_data[year]['ST_NUM'] = (parcel_data[year]['ST_NUM'].fillna("")).apply(lambda x: str(int(x)).strip() if type(x) == float else x.strip())

In [8]:
# Combine street number and street name data into address data as a tuple for analysis
for year in years:
    parcel_data[year]['ADDRESS'] = list(zip(parcel_data[year]['ST_NUM'], parcel_data[year]['ST_NAME'], parcel_data[year]['ZIPCODE']))

## Quantify condo conversions

In [9]:
num_records_by_address_by_year = {year: parcel_data[year]['ADDRESS'].value_counts().reset_index(name=f"num_records_{year}") for year in years}

In [10]:
num_records_by_address_by_year[2016].head()

,ADDRESS,num_records_2016
0,"(1313, WASHINGTON ST, 02118_)",535
1,"(42, EIGHTH ST, 02129_)",375
2,"(42529, WHITTIER PL, 02114_)",374
3,"(151, TREMONT ST, 02111_)",374
4,"(425, NEWBURY ST, 02115_)",329


In [11]:
# get condo conversions between prior_year and prior_year + 1
# returns dataframe with conversion data
def get_condo_conversions_df(prior_year): 
    records_df = pd.merge(num_records_by_address_by_year[prior_year], num_records_by_address_by_year[prior_year + 1], how="left", on='ADDRESS').set_index('ADDRESS')

    # condo conversions
    condos_df = records_df[(records_df[f"num_records_{prior_year}"] == 1) & (records_df[f"num_records_{prior_year + 1}"] > 1)]
    
    # prior dataframe, filtered by addresses that were converted
    prior_df = parcel_data[prior_year].set_index("ADDRESS")
    prior_df = prior_df[prior_df.index.isin(condos_df.index)].add_suffix('_prior').rename(columns={"ADDRESS_prior": "ADDRESS"})

    # post dataframe, filtered by addresses that were converted
    post_df = parcel_data[prior_year + 1].set_index("ADDRESS")
    post_df = post_df[post_df.index.isin(condos_df.index)].add_suffix('_post').rename(columns={"ADDRESS_post": "ADDRESS"})

    merged_df = prior_df.merge(post_df, how="left", on="ADDRESS")
    df = (condos_df
          .join(merged_df)
          .reset_index()
          .drop_duplicates(subset="ADDRESS")
          .rename(columns={f"num_records_{prior_year + 1}": "num_condo_units"})
          .drop(columns=[f"num_records_{prior_year}"])
          .set_index("ADDRESS"))
    df["num_condo_units"] = df.num_condo_units.astype(int)
    df["prior_year"] = prior_year
    df["post_year"] = prior_year + 1
    return df;


In [12]:
merged_df = pd.DataFrame()

In [13]:
for year in range(2015, 2024):
    merged_df = pd.concat([merged_df, get_condo_conversions_df(year)])

In [14]:
merged_df.head()

,num_condo_units,PID_prior,CM_ID_prior,ST_NUM_prior,ST_NAME_prior,UNIT_NUM_prior,ZIPCODE_prior,PTYPE_prior,LU_prior,LAND_VALUE_prior,...,BLDG_TYPE_prior,RES_FLOOR_prior,CD_FLOOR_prior,INT_COND_prior,EXT_COND_prior,OVERALL_COND_prior,HLF_BTH_prior,BDRM_COND_prior,HEAT_TYPE_prior,NUM_PARKING_prior
ADDRESS,,,,,,,,,,,,,,,,,,,,,
"(, MAYFIELD ST, 02125_)",2,1302046000_,NaN,,MAYFIELD ST,NaN,02125_,109.0,R3,178300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(, SPALDING ST, 02130_)",3,1103240000_,NaN,,SPALDING ST,NaN,02130_,132.0,RL,41600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(, W FOURTH ST, 02118_)",2,0801039000_,NaN,,W FOURTH ST,NaN,02118_,985.0,E,2067600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(, WHITFIELD ST, 02124_)",3,1700571000_,NaN,,WHITFIELD ST,NaN,02124_,130.0,RL,123800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(100, LEGENDS WY, 02114_)",3,0301925000_,NaN,100,LEGENDS WY,NaN,02114_,338.0,C,17007200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
merged_df = merged_df.reset_index()
merged_df["ADDRESS"] = merged_df["ADDRESS"].apply(lambda x: (x[0] + " " + x[1]).strip())

In [18]:
merged_df.keys()

Index(['ADDRESS', 'num_condo_units', 'PID_prior', 'CM_ID_prior',
       'ST_NUM_prior', 'ST_NAME_prior', 'UNIT_NUM_prior', 'ZIPCODE_prior',
       'PTYPE_prior', 'LU_prior', 'LAND_VALUE_prior', 'BLDG_VALUE_prior',
       'TOTAL_VALUE_prior', 'GROSS_TAX_prior', 'LAND_SF_prior',
       'YR_BUILT_prior', 'YR_REMODEL_prior', 'GROSS_AREA_prior',
       'LIVING_AREA_prior', 'NUM_FLOORS_prior', 'STRUCTURE_CLASS_prior',
       'TT_RMS_prior', 'BED_RMS_prior', 'FULL_BTH_prior', 'HALF_BTH_prior',
       'HEAT_TYP_prior', 'S_NUM_BLDG_prior', 'RES_UNITS_prior',
       'COM_UNITS_prior', 'RC_UNITS_prior', 'PID_post', 'CM_ID_post',
       'GIS_ID_post', 'ST_NUM_post', 'ST_NAME_post', 'UNIT_NUM_post',
       'ZIPCODE_post', 'PTYPE_post', 'LU_post', 'LAND_VALUE_post',
       'BLDG_VALUE_post', 'TOTAL_VALUE_post', 'GROSS_TAX_post', 'LAND_SF_post',
       'YR_BUILT_post', 'YR_REMODEL_post', 'GROSS_AREA_post',
       'LIVING_AREA_post', 'NUM_FLOORS_post', 'STRUCTURE_CLASS_post',
       'TT_RMS_post', 'BE

## Add geographic data

In [17]:
geo_parcels = pd.read_csv("./parcel_data/parcels_2022.csv")
geo_parcels.head()

/var/folders/77/mfz4g0815pzbmgddh8q96whm0000gn/T/ipykernel_18828/1143599125.py:1: DtypeWarning: Columns (1,4,6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  geo_parcels = pd.read_csv("./parcel_data/parcels_2022.csv")


,OBJECTID,MAP_PAR_ID,LOC_ID,POLY_TYPE,MAP_NO,SOURCE,PLAN_ID,LAST_EDIT,BND_CHK,NO_MATCH,TOWN_ID,Shape_STArea__,Shape_STLength__,ShapeSTArea,ShapeSTLength
0,1,107062500,F_799998_2955292,FEE,1,ASSESS,NaN,20191107,NaN,N,35,5.217380e+04,1420.659362,5.217380e+04,1420.659362
1,2,107069000,F_810842_2942136,FEE,1,ASSESS,NaN,20191107,NaN,N,35,1.700132e+06,5663.985895,1.700132e+06,5663.985895
2,3,107073100,F_794811_2942749,FEE,1,ASSESS,NaN,20191107,NaN,N,35,2.384763e+04,630.899280,2.384763e+04,630.899280
3,4,107071000,F_794865_2944326,FEE,1,ASSESS,NaN,20191107,NaN,N,35,2.042029e+06,6104.038098,2.042029e+06,6104.038098
4,5,107073000,F_795068_2942368,FEE,1,ASSESS,NaN,20191107,NaN,N,35,5.066375e+05,2891.186046,5.066375e+05,2891.186046


In [ ]:
merged_df = merged_df.set_index("GIS_ID_post").join(geo_parcels)

In [35]:
# merged_df.to_csv("condo_conversions.csv")